In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Generierung der Instanzen 

In [2]:
import random 
f = open("NEH_Instanzen_5_30.txt","w") 
Anzahl_Maschinen = 5 
Job_Anzahl = 30 
Instanzen = 50 
for m in range(Instanzen): 
    for i in range(1,Job_Anzahl+1):    
        f.write(str(i)) 
        for j in range(Anzahl_Maschinen): 
            f.write(",") 
            n = random.randint(5,20) 
            f.write(str(n)) 
         
        f.write("\n") 
f.close 

<function TextIOWrapper.close()>

# Implementierung von NEH-Heuristik 

In [3]:
import time 
startzeit = time.time()  
f = open("NEH_Instanzen_5_30.txt",'r') #Öffnet die Textdatei mit generierten Instanzen. 
Anzahl_Jobs = 30 #Repräsentiert die Anzahl der Aufträge und ist vom Benutzer einzugeben. 
Anzahl_Maschinen = 5 #Repräsentiert die Anzahl der Maschinen und ist vom Benutzer einzugeben. 
  
Auftraege = [] 
Liste_der_Bearbeitungszeiten = [] 
Sortierte_Auftragsliste = [] 

b = 0 
k = 0 
Sequenz = [] 
Lösung = [] 
Gespeicherte_Lösung = [] 

for j in range(Anzahl_Jobs): #Diese Schleife liest ins Programm zeilenweise die Bearbeitungszeiten der Aufträge ein. 
    Zeile = f.readline() 
    Auftrag = [] 
    Auftrag_als_string = Zeile.split(',') 
    for n in Auftrag_als_string:
        if n.strip():  # Only process non-empty strings
            Auftrag.append(int(n))

    Auftraege.append(Auftrag) 


for i in range(len(Auftraege)):  #Diese Schleife berechnet die Bearbeitungszeit einzelner Aufträge auf allen Maschinen.
    Auftragsbearbeitungszeit = 0  
    for j in range(1,Anzahl_Maschinen+1):  
        Auftragsbearbeitungszeit += Auftraege[i][j]  
        j=j+1
    Liste_der_Bearbeitungszeiten.append(Auftragsbearbeitungszeit) 

 
for a in range (len(Auftraege)): #Sortiert die Aufträge absteigend gemäß der Gesamtbearbeitungszeit. 
    l = [] 
    l.append(Auftraege[a]) 
    l.append(Liste_der_Bearbeitungszeiten[a]) 
    Sortierte_Auftragsliste.append(l) 
Sortierte_Auftragsliste.sort(key=lambda t : t[1], reverse = True)  
 
 
for i in range(0,len(Sortierte_Auftragsliste)):  #Diese Schleife fügt den nächsten Auftrag in die aktuelle Sequenz ein. 
    
    for pos in range(0,k+1):            #Diese Schleife ermöglicht den Wechsel der Positionen des Auftrages innerhalb aktueller Sequenz. 
        Teilsequenz = Sequenz.copy()        
        Teilsequenz.insert(pos,Sortierte_Auftragsliste[i])           
        Z = [0] * Anzahl_Maschinen #Erzeugt eine Liste mit einer Länge der Anzahl der Maschinen mit den Ausgangswerten gleich Null. 
 
        for b in range(len(Teilsequenz)) :   
            Z[0] = Z[0] + Teilsequenz[b][0][1] #Berechnet auftragsweise den Zielfunktionswert für die erste Maschine.  
             
 
 
            for c in range(1,Anzahl_Maschinen): #Berechnet auftragsweise den Zielfunktionswert für die weiteren Maschinen. 
                if(Z[c] < Z[c-1]): #Prüft ob der vorherige Arbeitsschritt abgeschlossen ist. Wenn es nicht zutrifft, dann wird die if-Bedingung ausgeführt und gemäß der Formel berechnet. 
                    Z[c] = Z[c-1] 
                    Z[c] = Z[c-1] + Teilsequenz[b][0][c+1] 
                else: 
                    Z[c] = Z[c] + Teilsequenz[b][0][c+1]                         
        if (pos == 0): #Bei der ersten Betrachtung des Auftrages wird der Zielfunktionswert mit dazugehöriger Sequenz übernommen. 
            B_Zielfunktionswert = Z[c] 
            B_Sequenz = Teilsequenz 
        if(pos > 0 and Z[c] < B_Zielfunktionswert): #Bei Erhöhung der Anzahl der Aufträge wird aktuellen Zielfunktionswert mit dem besten Zielfunktionswert vergliechen. 
                                          #Durch diesen Vergleich wählt der Algorithmus die beste Sequenz. 
            B_Sequenz = Teilsequenz 
            B_Zielfunktionswert = Z[c] 
        k = len(Teilsequenz) #k ist die Anzahl der möglichen Positionierungen eines Auftrages. 
    Sequenz = B_Sequenz         
 
 
laufzeit = time.time() - startzeit #Berechnet die Laufzeit des Programms. 
 
def Lösungsbereinigung(B_Sequenz): #Diese Funktion dient zur besseren Darstellung des Ergebnisses. 
    for a in range (len(B_Sequenz)): 
         Lösung.append(B_Sequenz[a][0][0]) 
    return Lösung 
Lösung = Lösungsbereinigung(B_Sequenz) 
 
print("Beste Reihenfolge:",Lösung,"Zielfunktionswert:",B_Zielfunktionswert,"Laufzeit:",laufzeit) 
 
def Lösung_speichern(B_Sequenz): 
    for a in range(len(B_Sequenz)): 
        Gespeicherte_Lösung.append(B_Sequenz[a][0]) 
    return Gespeicherte_Lösung 
Gespeicherte_Lösung = Lösung_speichern(B_Sequenz) 
print(Gespeicherte_Lösung) 
f.close

Beste Reihenfolge: [16, 25, 1, 21, 10, 7, 30, 23, 20, 6, 9, 8, 12, 26, 27, 19, 22, 28, 5, 17, 3, 4, 18, 15, 11, 24, 14, 29, 13, 2] Zielfunktionswert: 444 Laufzeit: 0.0232546329498291
[[16, 6, 8, 19, 20, 17], [25, 5, 10, 9, 5, 15], [1, 7, 14, 17, 12, 6], [21, 14, 11, 6, 15, 8], [10, 9, 13, 9, 8, 18], [7, 12, 8, 15, 16, 6], [30, 10, 19, 14, 20, 7], [23, 17, 16, 5, 8, 19], [20, 14, 14, 9, 13, 15], [6, 5, 17, 15, 9, 19], [9, 18, 14, 11, 19, 8], [8, 15, 14, 12, 11, 19], [12, 14, 6, 15, 19, 20], [26, 18, 13, 9, 12, 13], [27, 9, 15, 20, 17, 13], [19, 12, 19, 8, 11, 14], [22, 20, 6, 20, 20, 17], [28, 13, 18, 17, 17, 17], [5, 16, 16, 19, 10, 20], [17, 19, 20, 5, 19, 10], [3, 19, 16, 18, 18, 16], [4, 15, 17, 15, 17, 13], [18, 16, 20, 19, 12, 7], [15, 20, 13, 8, 13, 8], [11, 14, 11, 17, 9, 7], [24, 12, 19, 9, 8, 7], [14, 16, 13, 8, 16, 7], [29, 13, 14, 9, 9, 9], [13, 18, 7, 6, 5, 9], [2, 5, 11, 13, 8, 5]]


<function TextIOWrapper.close()>

# Implementierung von LAHC-Metaheuristik 

In [4]:
import random as rnd 
import time 
startzeit = time.time() 
Startlösung = [[3, 12, 5, 15, 20], [10, 6, 17, 16, 20], [8, 10, 14, 10, 11], [5, 12, 13, 10, 15], [4, 16, 8, 18, 18], [7, 19, 14, 17, 20], [2, 17, 20, 16, 14], [6, 16, 5, 14, 6], [1, 14, 15, 15, 7], [9, 18, 20, 6, 5]] 
  
C_xs = 171 
beste_Lösung = [] 
beste_Kosten = C_xs 
L = [C_xs]*20 #Eine Liste der Länge 20, wo die Lösungen einsortiert werden. 
i = 0 
v = 0 
Anzahl_Maschinen = 4 
verbesserung = 0 
 
b = 0 
k = 0 
Z = [] 
 
while k <= 100: #Abbruchsbedingung : k-Durchläufe 
    Z = [0] *Anzahl_Maschinen #Erzeugt eine Liste mit einer Länge der Anzahl der Maschinen mit den Ausgangswerten gleich Null. 
#In den nächsten Zeilen werden zwei zufällige Jobs miteinander getauscht. 
    m = rnd.randint(0,len(Startlösung)-1)  
    n = rnd.randint(0,len(Startlösung)-1) 
    if(m == n): 
        m = rnd.randint(0,len(Startlösung)-1) 
        n = rnd.randint(0,len(Startlösung)-1) 
    z = Startlösung.copy() 
    z[n],z[m] = z[m],z[n]   
     
    for b in range(len(Startlösung)): #Berechnet analog zur NEHHeuristik die Zielfunktionswerte für alle Maschinen 
        Z[0] = Z[0] + z[b][1]  
             
        for c in range(1,Anzahl_Maschinen):  
            if(Z[c] < Z[c-1]): 
                Z[c] = Z[c-1] 
                Z[c] = Z[c-1] + z[b][c+1] 
            else: 
                Z[c] = Z[c] + z[b][c+1]     
    v = i % len(L) #Index v für die Vergleichskosten 
    if(Z[c] <= L[v]): #Falls diese Bedingung stimmt, wird die Liste L mit besseren Zielfunktionswerten überschrieben. 
        Startlösung = z 
        L[v] = Z[c] 
        if(Z[c] < beste_Kosten): #Aktualisiert die global beste Lösung, falls Diese existiert. 
            beste_Lösung = z 
            beste_Kosten = Z[c] 
            verbesserung = verbesserung +1 
    k = k + 1 
    i = i + 1 
laufzeit = time.time() - startzeit 
print(beste_Kosten,verbesserung,laufzeit)

171 0 0.0029709339141845703
